In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
import torch.nn as nn
class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
            # nn.Softmax()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [3]:
# 定义超参和数据集等
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

EPOCH = 20
BATCH_SIZE = 100
LR = 1e-3
NUM_CLASSES = 10

# 预处理
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# 加载数据集
train_data = datasets.CIFAR10(root=".\data", train=True, transform=transform_train, download=False)
test_data = datasets.CIFAR10(root=".\data", train=False, transform=transform_test, download=False)

# 数据分批
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

In [4]:
# 定义获取准确度的函数

def get_accuracy(model, test_loader):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    mode = model.to(device)

    model.eval()  # 测试模式

    correct,total = 0,0
    for j,data in enumerate(test_loader):
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        #前向传播
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data,1)
        total =total+labels.size(0)
        correct = correct +(predicted == labels).sum().item()
        #准确率可视化
        # if  j%20 == 0:
        #     writer.add_scalar("Train/Accuracy", 100.0*correct/total, j)
            
    print('\n准确率为：{:.4f}%\n'.format(100.0*correct/total))
    return correct/total

In [5]:
# 准确度和loss的数组
import numpy as np

acc_pretrain, acc_retrain = [], []
loss_pretrain, loss_retrain = [], []

In [6]:
import torch.optim as optim

# 加载模型训练
model = AlexNet(10)

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 优化器
optimizer = optim.Adam(model.parameters(), lr=LR)

# 设置GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
mode = model.to(device)

model.train()

for epoch in range(EPOCH):
    for i,data in enumerate(train_loader):
        #取出数据及标签
        inputs,labels = data
        #数据及标签均送入GPU或CPU
        inputs,labels = inputs.to(device),labels.to(device)
        
        #前向传播
        outputs = model(inputs)
        #计算损失函数
        loss = criterion(outputs,labels)
        #清空上一轮的梯度
        optimizer.zero_grad()
        
        #反向传播
        loss.backward()
        #参数更新
        optimizer.step()

        #利用tensorboard，将训练数据可视化
        # if  i%50 == 0:
        #     writer.add_scalar("Train/Loss", loss.item(), epoch*len(train_loader)+i)
        print('it’s training...{}'.format(i))
    print('epoch{} loss:{:.4f}'.format(epoch+1,loss.item()))
    loss_pretrain.append(loss.item())  # loss数据存储
    # 保存模型
    torch.save(model, ".\data\model_para\AlexNetOrigin{}.pth".format(epoch))
    
    # 获取准确度
    acc_pretrain.append(get_accuracy(model, test_loader))
    

cuda:0
it’s training...0
it’s training...1
it’s training...2
it’s training...3
it’s training...4
it’s training...5
it’s training...6
it’s training...7
it’s training...8
it’s training...9
it’s training...10
it’s training...11
it’s training...12
it’s training...13
it’s training...14
it’s training...15
it’s training...16
it’s training...17
it’s training...18
it’s training...19
it’s training...20
it’s training...21
it’s training...22
it’s training...23
it’s training...24
it’s training...25
it’s training...26
it’s training...27
it’s training...28
it’s training...29
it’s training...30
it’s training...31
it’s training...32
it’s training...33
it’s training...34
it’s training...35
it’s training...36
it’s training...37
it’s training...38
it’s training...39
it’s training...40
it’s training...41
it’s training...42
it’s training...43
it’s training...44
it’s training...45
it’s training...46
it’s training...47
it’s training...48
it’s training...49
it’s training...50
it’s training...51
it’s training..

In [7]:
print(acc_pretrain)
print(loss_pretrain)

[0.4622, 0.6149, 0.6683, 0.7007, 0.7107, 0.7249, 0.7287, 0.7406, 0.7382, 0.7347, 0.7406, 0.7437, 0.7334, 0.7168, 0.7333, 0.7411, 0.7354, 0.7289, 0.7352, 0.7272]
[1.6647305488586426, 1.0485097169876099, 0.6927691102027893, 0.7509926557540894, 0.8289713859558105, 0.5829280018806458, 0.5992640852928162, 0.48749101161956787, 0.47670528292655945, 0.2571370601654053, 0.31140339374542236, 0.4050317704677582, 0.23686738312244415, 0.46028292179107666, 0.24474219977855682, 0.27159062027931213, 0.25715142488479614, 0.15900976955890656, 0.13411583006381989, 0.25872671604156494]


In [4]:
# 聚类算法

import random
import pandas as pd
import numpy as np
import math


# 计算距离
def Dis(dataSet, centroids, k):
    # 处理质心
    # 如果之前分类的个数不够k类
    if len(centroids) < k:
        centroids = np.append(centroids, random.sample(list(dataSet), k-len(centroids)), axis=0)
    
    # 处理节点
    clalist=[]
    for data in dataSet:
        #(np.tile(a,(2,1))就是把a先沿x轴复制1倍，即没有复制，仍然是 [0,1,2]。 再把结果沿y方向复制2倍得到array([[0,1,2],[0,1,2]]))
        diff = np.tile(data, (k, 1)) 
        mul_Diff = np.multiply(diff, centroids)
        mul_Dist = np.sum(mul_Diff, axis=1)   #和  (axis=1表示行)
        clalist.append(mul_Dist) 
    clalist = np.array(clalist)  #返回一个每个点到质点的距离len(dateSet)*k的数组
    return clalist 


# 计算质心
def classify(dataSet, centroids, k):
    # 计算样本到质心的距离
    clalist = Dis(dataSet, centroids, k)
    # 分组并计算新的质心
    minDistIndices = np.argmax(clalist, axis=1)    #axis=1 表示求出每行的最小值的下标
    newCentroids = pd.DataFrame(dataSet).groupby(minDistIndices).mean() #DataFramte(dataSet)对DataSet分组，groupby(min)按照min进行统计分类，mean()对分类结果求均值
    newCentroids = newCentroids.values

    # 对新质心，也分配成1-value_sum的形式，否则会出现小数
    for centro in newCentroids:
        # centro是一个一维向量
        sorted_data=np.argsort(centro)  # 排序信息
        value = 1
        for valueIndex in sorted_data:
            centro[valueIndex] = value
            value += 1
    
    # 计算变化量
    # 有可能新分类个数不够k
    if len(newCentroids) != len(centroids):
        changed = 1  # 肯定有变化
    else:
        changed = newCentroids - centroids # 有可能没变化 

    return changed, newCentroids


#确定初始中心点
def euler_distance(point1: list, point2: list) -> float:
    """
    计算两点之间的欧拉距离，支持多维
    distance = 0.0
    for a, b in zip(point1, point2):
        distance += math.pow(a - b, 2)
    return math.sqrt(distance)
    """
    distance = 0.0
    for a, b in zip(point1, point2):
        distance += a*b
    return distance
    

def get_closest_dist(point, centroids):
    min_dist = math.inf  # 初始设为无穷大
    for i, centroid in enumerate(centroids):
        dist = euler_distance(centroid, point)
        if dist < min_dist:
            min_dist = dist
    return min_dist


def kpp_centers(data_set: list, k: int) -> list:
    """
    从数据集中返回 k 个对象可作为质心
    """
    cluster_centers = []
    cluster_centers.append(random.choice(data_set))
    d = [0 for _ in range(len(data_set))]
    for _ in range(1, k):
        total = 0.0
        for i, point in enumerate(data_set):
            d[i] = get_closest_dist(point, cluster_centers) # 与最近一个聚类中心的距离
            total += d[i]
        total *= random.random()
        for i, di in enumerate(d): # 轮盘法选出下一个聚类中心；
            total -= di
            if total > 0:
                continue
            cluster_centers.append(data_set[i])
            break
    return cluster_centers


# 使用k-means分类
def kmeans(dataSet, k):
    # 将dataSet预处理成为算距离需要使用的重要程度矩阵
    valueSet = np.zeros(dataSet.shape, dtype=int)  # 初始矩阵
    for index in range(len(dataSet)):
        data = dataSet[index]
        value = valueSet[index]
        sorted_data=list(map(abs,data))  # 绝对值
        sorted_data=np.argsort(sorted_data)  # 排序信息
        i = 1  # 对于越小的值，分配的i越小
        for valueIndex in sorted_data:
            value[valueIndex] = i
            i += 1

    # 随机取质心
    # centroids = random.sample(dataSet, k)
    centroids=kpp_centers(valueSet, k)
    
    # 更新质心 直到变化量全为0
    i=100
    changed, newCentroids = classify(valueSet, centroids, k)
    # while(i): #while np.any(changed != 0)
    while np.any(changed != 0) and i > 0:
        changed, newCentroids = classify(valueSet, newCentroids, k)
        i=i-1
        print("第{}次迭代".format(100-i))
 
    centroids = sorted(newCentroids.tolist())   #tolist()将矩阵转换成列表 sorted()排序
 
    clalist = Dis(valueSet, centroids, k) 
    minDistIndices = np.argmax(clalist, axis=1)  
    return minDistIndices


def getCluster(input, clusters_num):
    # 对卷积层聚类为4维，对全连接层聚类为2维
    if len(input.shape) == 2:  # 如果是全连接层
        fcValues = input.detach().cpu().numpy()  # 转成numpy
        # input.shape[1]是聚类基本单位的数据个数
        clusterIndex = kmeans(fcValues, clusters_num)  # 分类
    elif len(input.shape) == 4:  # 卷积层
        kernel_size = input.shape[3]  # 卷积核尺寸
        preShape = input.shape[:2]  # 四维数据的前两维
        inputCut = input.view(preShape[0]*preShape[1], kernel_size*kernel_size)  # 降维后的数据，四维到二维
        convValues = inputCut.detach().cpu().numpy()  # 转成numpy
        clusterIndex = kmeans(convValues, clusters_num)  # 分类
        clusterIndex.resize(preShape)
    else:
        clusterIndex = None
    
    return clusterIndex

In [5]:
# 裁剪算法


import torch.nn.utils.prune as prune
import pandas as pd
import numpy as np

def scp_upgrade(kernel,old_scp):
    old_scp+=np.abs(kernel.cpu().detach().numpy())
    return old_scp

def scp_binaeryzation(scps,C):
    if len(scps.shape)==3:
        for r in np.arange(0,scps.shape[0]):
            series=pd.Series(scps[r].ravel())
            rank_info=series.rank()
            for i in np.arange(0,scps[r].shape[0]):
                for j in np.arange(0,scps[r].shape[1]):
                    index=i*scps[r].shape[0]+j
                    if(rank_info[index]<=C):
                        scps[r][i][j]=0
                    else:
                        scps[r][i][j]=1
    elif len(scps.shape)==2:
        for r in np.arange(0,scps.shape[0]):
            series=pd.Series(scps[r].ravel())
            rank_info=series.rank()
            for i in np.arange(0,scps[r].shape[0]):
                    index=i
                    if(rank_info[index]<=C):
                        scps[r][i]=0
                    else:
                        scps[r][i]=1

class PatternPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE= "unstructured"
    
    def __init__(self, custers_num, cut_num, pruning_type):
        self.clusters_num=custers_num
        self.cut_num=cut_num
        self.pruning_type=pruning_type
        prune.BasePruningMethod.__init__(self)
        
    def compute_mask(self, t, default_mask):
        mask=default_mask.clone()#复制一个mask大小等于当前层的filter
        if self.pruning_type=='conv':
            scps=np.zeros(self.clusters_num*default_mask.shape[-1]*default_mask.shape[-1])#复制num个scp,表示每一个卷积族的pattern
            scps.resize(self.clusters_num,default_mask.shape[-1],default_mask.shape[-1])
         
            clusters=getCluster(t,self.clusters_num)#输入当前层的filter，获得其聚类信息           
            # print(clusters)
            
            for i in np.arange(0,clusters.shape[0]):#遍历所有kernel,计算所有cluster的scp
                for j in np.arange(0,clusters.shape[1]):
                    scp_upgrade(t[i][j],scps[clusters[i][j]])
           
            scp_binaeryzation(scps,self.cut_num)#根据scp二值化获得真正的pattern
            print(scps)
            
            for i in  np.arange(0,clusters.shape[0]):#根据scp和每个kernel的族编号得到最终的mask
                for j in  np.arange(0,clusters.shape[1]):
                        mask[i][j]=torch.from_numpy(scps[clusters[i][j]])     
                        
        elif self.pruning_type=='full':

            scps=np.zeros(self.clusters_num*default_mask.shape[-1])
            scps.resize(self.clusters_num,default_mask.shape[-1])            
            clusters=getCluster(t,self.clusters_num)

            # print(clusters)
            
            for i in np.arange(0,clusters.shape[0]):
                scp_upgrade(t[i],scps[int(clusters[i])])
           
            scp_binaeryzation(scps,self.cut_num)#根据scp二值化获得真正的pattern
            print(scps)
            
            for i in  np.arange(0,clusters.shape[0]):#根据scp和每个kernel的族编号得到最终的mask
                mask[i]=torch.from_numpy(scps[int(clusters[i])]) 
                
          
        return mask

In [8]:
# 加载之前训练好的权重

phtfile = ".\data\model_para\AlexNetOrigin19.pth"
model = torch.load(phtfile)

In [9]:
# 测试训练好的准确率
get_accuracy(model, test_loader)

cuda:0

准确率为：72.7200%



0.7272

In [17]:
# 输出卷积层和全连接层的信息

# phtfile = ".\data\model_para\AlexNetOrigin9.pth"
# model = torch.load(phtfile)

for mode in model.children():
    for layer in mode.modules():
        if isinstance(layer, nn.Conv2d):
            layerName = "卷积层"
        elif isinstance(layer, nn.Linear): 
        # and layer.out_features != NUM_CLASSES:
            layerName = "全连接层"
        else:
            continue;
        print("{}的尺寸为：".format(layerName), layer.weight.shape)
        values = layer.weight.detach().cpu().numpy()  # numpy格式
        nonzero = np.count_nonzero(values, axis=None)
        print("该{}共有{}个权重，非零元素个数为{}".format(layerName, values.size, nonzero))


卷积层的尺寸为： torch.Size([96, 3, 11, 11])
该卷积层共有34848个权重，非零元素个数为34848
卷积层的尺寸为： torch.Size([256, 96, 5, 5])
该卷积层共有614400个权重，非零元素个数为614400
卷积层的尺寸为： torch.Size([384, 256, 3, 3])
该卷积层共有884736个权重，非零元素个数为884736
卷积层的尺寸为： torch.Size([384, 384, 3, 3])
该卷积层共有1327104个权重，非零元素个数为1327104
卷积层的尺寸为： torch.Size([256, 384, 3, 3])
该卷积层共有884736个权重，非零元素个数为884736
全连接层的尺寸为： torch.Size([4096, 9216])
该全连接层共有37748736个权重，非零元素个数为37748736
全连接层的尺寸为： torch.Size([4096, 4096])
该全连接层共有16777216个权重，非零元素个数为16777216
全连接层的尺寸为： torch.Size([10, 4096])
该全连接层共有40960个权重，非零元素个数为40960


In [20]:
# 定义剪枝的超参
CONV_CLUSTER_SUM = 6
FC_CLUSTER_SUM = 10
CONV_PRUNE_RATIO = 0.75
FC_PRUNE_RATIO = 0.25

for mode in model.children():
    for layer in mode.modules():
        if isinstance(layer, nn.Conv2d):
            layerName = 'conv'
            cutSum = int(layer.weight.shape[-1]*layer.weight.shape[-2]*CONV_PRUNE_RATIO)
            print("开始聚类{}".format(layerName))
            PatternPruningMethod.apply(layer, 'weight', CONV_CLUSTER_SUM, cutSum, layerName)
        # 最后一层全连接层不减
        elif isinstance(layer, nn.Linear):
        # and layer.out_features != 10:  # NUM_CLASSES
            layerName = 'full'
            cutSum = int(layer.weight.shape[-1]*CONV_PRUNE_RATIO)
            print("开始聚类{}".format(layerName))
            PatternPruningMethod.apply(layer, 'weight', FC_CLUSTER_SUM, cutSum, layerName)
        else:
            continue

        layer._forward_pre_hooks
        print("聚类结束")
            # break

开始聚类conv
第1次迭代
第2次迭代
第3次迭代
第4次迭代
第5次迭代
第6次迭代
第7次迭代
第8次迭代
第9次迭代
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
  [0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.]
  [0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
  [1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0.]
  [1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
  [1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]

 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
  [1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
  [1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
  [1. 0.

In [21]:
# 聚类并且有mask的网络结构存起来
torch.save(model, ".\data\model_para\AlexNetOrigin_prune_initial.pth")

In [10]:
# 实验一下  聚类结束有mask的网络准确率
model = torch.load(".\data\model_para\AlexNetOrigin_prune_initial.pth")
get_accuracy(model, test_loader)

cuda:0

准确率为：10.8400%



0.1084

In [11]:
# 查看剪枝效果
for mode in model.children():
    for layer in mode.modules():
        if isinstance(layer, nn.Conv2d):
            layerName = "卷积层"
        elif isinstance(layer, nn.Linear): 
        # and layer.out_features != NUM_CLASSES:
            layerName = "全连接层"
        else:
            continue;
        print("{}的尺寸为：".format(layerName), layer.weight.shape)
        values = layer.weight.detach().cpu().numpy()  # numpy格式
        nonzero = np.count_nonzero(values, axis=None)
        print("该{}共有{}个权重，非零元素个数为{}".format(layerName, values.size, nonzero))

卷积层的尺寸为： torch.Size([96, 3, 11, 11])
该卷积层共有34848个权重，非零元素个数为8928
卷积层的尺寸为： torch.Size([256, 96, 5, 5])
该卷积层共有614400个权重，非零元素个数为172032
卷积层的尺寸为： torch.Size([384, 256, 3, 3])
该卷积层共有884736个权重，非零元素个数为294912
卷积层的尺寸为： torch.Size([384, 384, 3, 3])
该卷积层共有1327104个权重，非零元素个数为442368
卷积层的尺寸为： torch.Size([256, 384, 3, 3])
该卷积层共有884736个权重，非零元素个数为294912
全连接层的尺寸为： torch.Size([4096, 9216])
该全连接层共有37748736个权重，非零元素个数为9437184
全连接层的尺寸为： torch.Size([4096, 4096])
该全连接层共有16777216个权重，非零元素个数为4194304
全连接层的尺寸为： torch.Size([10, 4096])
该全连接层共有40960个权重，非零元素个数为10240


In [12]:
# 进行重训练
import torch.optim as optim

# 重训练轮数
RETRAIN_EPOCH = 20

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 优化器
optimizer = optim.Adam(model.parameters(), lr=LR)

# 设置GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
mode = model.to(device)

model.train()

for epoch in range(RETRAIN_EPOCH):
    for i,data in enumerate(train_loader):
        #取出数据及标签
        inputs,labels = data
        #数据及标签均送入GPU或CPU
        inputs,labels = inputs.to(device),labels.to(device)
        
        #前向传播
        outputs = model(inputs)
        #计算损失函数
        loss = criterion(outputs,labels)
        #清空上一轮的梯度
        optimizer.zero_grad()
        
        #反向传播
        loss.backward()
        #参数更新
        optimizer.step()

        #利用tensorboard，将训练数据可视化
        # if  i%50 == 0:
        #     writer.add_scalar("Train/Loss", loss.item(), epoch*len(train_loader)+i)
        print('it’s training...{}'.format(i))
    print('epoch{} loss:{:.4f}'.format(epoch+1,loss.item()))
    loss_retrain.append(loss.item())  # loss数据存储
    
    # 保存模型
    torch.save(model, ".\data\model_para_prune\AlexNetOrigin{}.pth".format(epoch))
    
    # 获取准确度
    acc_retrain.append(get_accuracy(model, test_loader))
    

cuda:0
it’s training...0
it’s training...1
it’s training...2
it’s training...3
it’s training...4
it’s training...5
it’s training...6
it’s training...7
it’s training...8
it’s training...9
it’s training...10
it’s training...11
it’s training...12
it’s training...13
it’s training...14
it’s training...15
it’s training...16
it’s training...17
it’s training...18
it’s training...19
it’s training...20
it’s training...21
it’s training...22
it’s training...23
it’s training...24
it’s training...25
it’s training...26
it’s training...27
it’s training...28
it’s training...29
it’s training...30
it’s training...31
it’s training...32
it’s training...33
it’s training...34
it’s training...35
it’s training...36
it’s training...37
it’s training...38
it’s training...39
it’s training...40
it’s training...41
it’s training...42
it’s training...43
it’s training...44
it’s training...45
it’s training...46
it’s training...47
it’s training...48
it’s training...49
it’s training...50
it’s training...51
it’s training..

In [ ]:
print(acc_retrain)
print(loss_retrain)

In [6]:
import torch

model = torch.load(".\data\model_para_prune\AlexNetOrigin19.pth")

# 打印一下模型的修剪权重
for mode in model.children():
    for layer in mode.modules():
        if isinstance(layer, nn.Conv2d):
            layerName = "卷积层"
        elif isinstance(layer, nn.Linear): 
        # and layer.out_features != NUM_CLASSES:
            layerName = "全连接层"
        else:
            continue;
        print("{}的尺寸为：".format(layerName), layer.weight.shape)
        values = layer.weight.detach().cpu().numpy()  # numpy格式
        nonzero = np.count_nonzero(values, axis=None)
        print("该{}共有{}个权重，非零元素个数为{}".format(layerName, values.size, nonzero))

卷积层的尺寸为： torch.Size([96, 3, 11, 11])
该卷积层共有34848个权重，非零元素个数为8928
卷积层的尺寸为： torch.Size([256, 96, 5, 5])
该卷积层共有614400个权重，非零元素个数为172032
卷积层的尺寸为： torch.Size([384, 256, 3, 3])
该卷积层共有884736个权重，非零元素个数为294912
卷积层的尺寸为： torch.Size([384, 384, 3, 3])
该卷积层共有1327104个权重，非零元素个数为442368
卷积层的尺寸为： torch.Size([256, 384, 3, 3])
该卷积层共有884736个权重，非零元素个数为294912
全连接层的尺寸为： torch.Size([4096, 9216])
该全连接层共有37748736个权重，非零元素个数为9437184
全连接层的尺寸为： torch.Size([4096, 4096])
该全连接层共有16777216个权重，非零元素个数为4194304
全连接层的尺寸为： torch.Size([10, 4096])
该全连接层共有40960个权重，非零元素个数为10240
